# 02 Performance Analysis

## 1. Measure computation time

首先，我们讨论一下如何衡量一个函数的运行时间。

我们已菲波纳西数列为例，函数如下。

In [3]:
def fibonacci(n): 
    if n<0: 
        print("Incorrect input") 
    # First Fibonacci number is 0 
    elif n==1: 
        return 0
    # Second Fibonacci number is 1 
    elif n==2: 
        return 1
    else: 
        return fibonacci(n-1) + fibonacci(n-2) 

### 方法1: 使用`time.time()` 函数

This is a quick and dirty mothod.

In [16]:
import time

In [17]:
t_start = time.time()
result = fibonacci(35)
t_end = time.time()

print('result: ', result)
print('time consumption: ', t_end - t_start)

result:  5702887
time consumption:  3.394181728363037


In [21]:
from functools import wraps

def timefn(fn):
    @wraps(fn)
    def measure_time(*args, **kwargs):
        t_start = time.time()
        result = fn(*args, **kwargs)
        t_end = time.time()
        
        print('@timefn: {} took {} seconds.'.format(fn.__name__, str(t_end - t_start)))
        return result
    
    return measure_time

### 方法2: 使用修饰器

In [22]:
@timefn
def fibonacci_2(n): 
    if n<0: 
        print("Incorrect input") 
    # First Fibonacci number is 0 
    elif n==1: 
        return 0
    # Second Fibonacci number is 1 
    elif n==2: 
        return 1
    else: 
        return fibonacci(n-1) + fibonacci(n-2) 

In [23]:
t_start = time.time()
result = fibonacci_2(35)
t_end = time.time()

print('result: ', result)
print('time consumption: ', t_end - t_start)

@timefn: fibonacci_2 took 3.280791997909546 seconds.
result:  5702887
time consumption:  3.2809600830078125
